# This example is a duplication of [Mason Victor](http://www.recursionpharma.com/mason.html)'s [lightning talk at SciPy 2016](https://youtu.be/sv9S-25XKe4?t=54m40s) 

In [1]:
import pandas as pd
import numpy as np
import os

import bokeh.plotting as bop
from bokeh.models import ColumnDataSource, HoverTool, WheelZoomTool
bop.output_notebook()

Loading BokehJS ...

In [2]:
import keras
import theano
from keras.datasets import mnist as mnist_data
from keras.layers import Input
from keras.layers.convolutional import (
    Convolution2D, ZeroPadding2D,
    AveragePooling2D, MaxPooling2D
)
from keras.layers.core import Dense, Activation, Flatten, Lambda, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.models import Model
from keras import backend as K
from keras.utils.np_utils import to_categorical

theano.config.compute_test_value = 'off'

Using TensorFlow backend.
Using gpu device 0: GeForce GTX 750 Ti (CNMeM is enabled with initial size: 65.0% of memory, cuDNN 5110)
/home/schowell/data/myPrograms/anaconda3/envs/k35/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


# Traditional Dimensionality Reduction

## PCA

In [3]:
from sklearn.decomposition import PCA

In [4]:
(X_train, y_train), (X_test, y_test) = mnist_data.load_data()

X_test = np.reshape(X_test, (10000, 784))

X_test_df = pd.DataFrame(X_test)
X_test_df = X_test_df[X_test_df.columns[X_test_df.std() > 0.0]]
X_test_df = (X_test_df - X_test_df.mean()) / X_test_df.std()
X_test = X_test_df.values

pcaMod = PCA(n_components=2)
representations=pcaMod.fit_transform(X_test)

In [5]:
from bokeh.palettes import Category10_10 as palette

In [7]:
representations.shape

(10000, 2)

In [9]:
len(colors)

10000

In [11]:
img_urls = ['MNIST_images/img%d.jpg' % x for x in range(len(X_test))]

In [14]:
color_map = dict(zip(np.arange(10), palette))
colors = [color_map[i] for i in y_test]
img_urls = ['MNIST_images/img%d.jpg' % x for x in range(len(X_test))]

cds = ColumnDataSource(data=dict(x=representations[:, 0],
                                 y=representations[:, 1],
                                 color=colors,
                                 img_url=img_urls))

hover = HoverTool(tooltips="""<img src="@img_url" heigh="42" wiwdth="42"></img>""")
p = bop.figure(tools=[hover])
p.circle(x='x', y='y', size=5, alpha=0.5, color='color', source=cds)
bop.show(p)

# Deep Learning Dimensionality Reduction
![title](https://www.researchgate.net/profile/Luis_Ferraz/publication/269935583/figure/fig1/AS:294994507255823@1447343579348/Figure-1-Left-Schematic-of-a-siamese-network-where-pairs-of-input-patches-are.png)
(see https://www.researchgate.net/figure/269935583_fig1_Figure-1-Left-Schematic-of-a-siamese-network-where-pairs-of-input-patches-are.png)

In [ ]:
def contrastive_loss(y_true, y_pred):
    '''
    Contrastive loss from Hadsell et al. 2006
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''